## Hi all as we know credit card fraud detection will have a imbalanced data i.e having more number of normal class than the number of fraud class

### In this I will use Basic method of handling imbalance data which are
 ** This all I have done by using Analytics Vidya's blog please find the link [Analytics Vidya](https://www.analyticsvidhya.com/blog/2017/03/read-commonly-used-formats-using-python/)  **

Undersampling:- it means taking the less number of majority class (In our case taking less number of Normal transactions so that our new data will be balanced

Oversampling: it means using replicating the data of minority class (fraud class) so that we can have a balanced data

SMOTE: it is also a type of oversampling but in this we will make the synthetic example of Minority data and will give as a balanced data

First I will start with the Undersampling and will try to classify using these Models

1. Decision Tree Classifier/ Random Forest Classifier

2. Logistic regression

3. SVM

4. XGboost

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output

In [2]:
import pandas as pd # to import csv and for data manipulation
import matplotlib.pyplot as plt # to plot graph
import numpy as np # for linear algebra
import datetime # to dela with date and time
%matplotlib inline
from sklearn.preprocessing import StandardScaler # for preprocessing the data
from sklearn.ensemble import RandomForestClassifier # Random forest classifier
from sklearn.tree import DecisionTreeClassifier # for Decision Tree classifier
from sklearn.svm import LinearSVC as SVC # for SVM classification
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split # to split the data
from sklearn.cross_validation import KFold # For cross vbalidation
from sklearn.model_selection import GridSearchCV # for tunnig hyper parameter it will use all combination of given parameters
from sklearn.model_selection import RandomizedSearchCV # same for tunning hyper parameter but will use random combinations of parameters
from sklearn.metrics import confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
import warnings
warnings.filterwarnings('ignore')

/home/leshare/softwares/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
data = pd.read_csv("../data/real/creditcard.csv",header = 0)

In [4]:
# now let us check in the number of Percentage
Count_Normal_transacation = len(data[data["Class"]==0]) # normal transaction are repersented by 0
Count_Fraud_transacation = len(data[data["Class"]==1]) # fraud by 1
Percentage_of_Normal_transacation = 1.*Count_Normal_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of normal transacation is",Percentage_of_Normal_transacation*100)
Percentage_of_Fraud_transacation= 1.*Count_Fraud_transacation/(Count_Normal_transacation+Count_Fraud_transacation)
print("percentage of fraud transacation",Percentage_of_Fraud_transacation*100)

('percentage of normal transacation is', 99.82725143693798)
('percentage of fraud transacation', 0.1727485630620034)


1. Hence in data there is only 0.17 % are the fraud transcation while 99.83 are valid transcation
2. So now we have to do resampling of this data
3. before doing resampling lets have look at the amount related to valid transcation and fraud transcation

### No-sampling

In [5]:
#now let us a define a function for make undersample data with different proportion
#different proportion means with different proportion of normal classes of data
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import precision_recall_curve, auc, f1_score, precision_recall_fscore_support


def evaluate_auc_prc(y, pred):
    precision, recall, thresholds = precision_recall_curve(y, pred)
    aucprc = auc(recall, precision)
    #print 'AUPRC:{}'.format(aucprc)
    #plt.title('Precision Recall Curve')
    #plt.plot(precision, recall, 'b',label='AUC = %0.2f'% aucprc)
    #plt.legend(loc='lower right')
    #plt.ylabel('Precision')
    #plt.xlabel('Recall')
    #plt.show()
    return aucprc

aucprc_scorer = make_scorer(evaluate_auc_prc, greater_is_better=True)

def evaluate_f1(y, y_pred, pos_label=1):
    f1 = f1_score(y, y_pred, pos_label=1)
    precision, recall, f1, support = precision_recall_fscore_support(y, y_pred, pos_label=1)
    accuracy = accuracy_score(y, y_pred)
    print('precision:{} recall:{} F1: {}'.format(precision, recall, f1))

def data_split(x, test_size=0.4, seed=1): # preparing data for training and testing as we are going to use different data 
    #again and again so make a function
    x_features= x.ix[:,x.columns != "Class"]
    x_labels=x.ix[:,x.columns=="Class"]
    x_features_train,x_features_test,x_labels_train,x_labels_test = train_test_split(x_features,x_labels,
                                                                                     test_size=test_size, 
                                                                                     random_state=seed)
    x_features_train.index = np.arange(len(x_features_train))
    x_labels_train.index = np.arange(len(x_labels_train))
    x_features_test.index = np.arange(len(x_features_test))
    x_labels_test.index = np.arange(len(x_labels_test))
    print("length of training data")
    print(len(x_features_train))
    print("length of test data")
    print(len(x_features_test))
    return(x_features_train,x_features_test,x_labels_train,x_labels_test)

In [6]:
# before starting we should standridze our amount column
data["Normalized Amount"] = StandardScaler().fit_transform(data['Amount'].reshape(-1, 1))
data.drop(["Time","Amount"],axis=1,inplace=True)

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score

x_train = data.ix[:,data.columns != "Class"].values
y_train = data.ix[:,data.columns=="Class"].values[:, 0]
print 'train data size:{}'.format(len(y_train))

train data size:284807


In [10]:
import pickle
# pickle.dump({'x':x_train, 'y':np.eye(2)[y_train.astype('int32')]}, open('../data/real/creditcard/train.pkl', 'wb+'))
idxes = np.arange(len(x_train))
np.random.seed(1234)
np.random.shuffle(idxes)
x_train = x_train[idxes]
y_train = y_train[idxes]

In [11]:
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.linear_model import LogisticRegression as LR
from sklearn.svm import LinearSVC as SVM
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier as DT
from time import time

In [18]:
# Choose classifer with cross validation
# Supervised Model: LR, SVM (linear), SVM (rbf), DT, KNN, MLP
# Dataset: original
from sklearn.neural_network import MLPClassifier as MLP
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef

# LR
# best_c = 0
# best_reward = 0
# for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
#     clf = LR(C=c, random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for LR:{}'.format(best_c)
# print 'best_reward for LR:{}'.format(best_reward)

# SVM (linear)
# best_c = 0
# best_reward = 0
# for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
#     clf = SVM(C=c, random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for SVM:{}'.format(best_c)
# print 'best_reward for SVM:{}'.format(best_reward)

# SVM (rbf)
# best_c = 0
# best_reward = 0
# for c in [1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3]:
#     clf = SVC(C=c, kernel='rbf', random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for SVC:{}'.format(best_c)
# print 'best_reward for SVC:{}'.format(best_reward)

# DT 
# best_c = 0
# best_reward = 0
# for c in [1, 2, 3, 4, 5, 6, 7, 8]:
#     clf = DT(max_depth=c, random_state=0)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for DT:{}'.format(best_c)
# print 'best_reward DT:{}'.format(best_reward)

# KNN 
# best_c = 0
# best_reward = 0
# for c in [1, 2, 3, 4, 5, 6, 7, 8]:
#     clf = KNN(n_neighbors=c)
#     reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)).mean()
#     if reward > best_reward:
#         best_reward = reward
#         best_c = c
# print 'best_c for KNN:{}'.format(best_c)
# print 'best_reward KNN:{}'.format(best_reward)

# MLP 
best_c = 0
best_reward = 0
for c in [1, 2, 3, 4, 5, 6, 7, 8]:
    clf = MLP(hidden_layer_sizes=[5, c], random_state=0)
    reward = np.array(cross_val_score(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)).mean()
    if reward > best_reward:
        best_reward = reward
        best_c = c
print 'best_c for MLP:{}'.format(best_c)
print 'best_reward MLP:{}'.format(best_reward)
# 0.82582, c=5

best_c for MLP:6
best_reward MLP:0.823118981313


In [256]:
# Choose model with cross validation
# Supervised Model: DT
# Dataset: original
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.svm import SVC
from sklearn.metrics import matthews_corrcoef


best_c = 0
best_reward = 0
best_train_reward = 0
for c in np.range(2, 10):
    clf = DT(max_depth=4)
    rewards = cross_validate(clf, x_train, y_train, cv=5, n_jobs=5, scoring=aucprc_scorer)
    train_reward = np.array(rewards['train_score']).mean()
    reward = np.array(rewards['test_score']).mean()
    if reward > best_reward:
        best_reward = reward
        best_train_reward = train_reward
        best_c = c
print 'best_c:{}'.format(best_c)
print 'best_reward:{}'.format(best_reward)
print 'best_train_reward:{}'.format(best_train_reward)

best_c:1.0
best_reward:0.806395661911
best_train_reward:0.846036788206


In [276]:
# Test performance trianed on the original labeled dataset
clf = DT(max_depth=4)
clf.fit(x_train, y_train)
preds = clf.predict_proba(x_train)[:, 1]
print evaluate_auc_prc(y_train, preds)

0.848909682323


In [294]:
from collections import Counter
Counter(y_train[:, 0])

Counter({0: 284315, 1: 492})

In [289]:
# Choose under-sampling ratio for different under-sampling methods
# Supervised Model: DT
# Test under-sampling methods
from imblearn.under_sampling import (NearMiss, RandomUnderSampler, EditedNearestNeighbours, 
                                     CondensedNearestNeighbour, ClusterCentroids, TomekLinks,
                                     RepeatedEditedNearestNeighbours, AllKNN)
from imblearn.pipeline import make_pipeline
from collections import Counter

max_reward = 0
max_i = 0
rewards = []
train_rewards = []
train_x = x_train
train_y = y_train

best_train_reward = 0
for i in np.arange(400, 550, 10):
    print 'i:{}'.format(i)
    pos_num = (train_y == 1).sum()
    neg_num = (train_y == 0).sum()
    sampler = RandomUnderSampler(ratio={0:i*pos_num})
    train_x_s, train_y_s = sampler.fit_sample(train_x, train_y)
    clf.fit(train_x_s, train_y_s)
    preds = clf.predict_proba(train_x)[:, 1]
    train_reward = evaluate_auc_prc(train_y, preds)
    if best_train_reward < train_reward:
        best_train_reward = train_reward    
        max_i = i
    print 'reward:{}'.format(reward)
print 'best_train_reward:{}'.format(best_train_reward)
print 'best_i:{}'.format(max_i)

Counter({0: 196800, 1: 492})
Counter({0: 201720, 1: 492})
Counter({0: 206640, 1: 492})
Counter({0: 211560, 1: 492})
Counter({0: 216480, 1: 492})
Counter({0: 221400, 1: 492})
Counter({0: 226320, 1: 492})
Counter({0: 231240, 1: 492})
Counter({0: 236160, 1: 492})
Counter({0: 241080, 1: 492})
Counter({0: 246000, 1: 492})
Counter({0: 250920, 1: 492})
Counter({0: 255840, 1: 492})
Counter({0: 260760, 1: 492})
Counter({0: 265680, 1: 492})
best_train_reward:0.85466396156
best_i:490


In [21]:
# Perform EasyEnsemble and BalanceCascade
from imblearn.ensemble import EasyEnsemble, BalanceCascade

train_x = x_train
train_y = y_train
pos_num = (train_y == 1).sum()
neg_num = (train_y == 0).sum()

for sampler, name in zip([EasyEnsemble, BalanceCascade],
                         ['EasyEnsemble']):
    max_i = 1
    best_reward = -1
    for i in [490]:
        sample = sampler(ratio={0:i*pos_num}, replacement=False, n_subsets=10)
        train_x_s, train_y_s = sample.fit_sample(x_train, y_train)
        preds = None
        for x, y in zip(train_x_s, train_y_s):
            clf.fit(x, y)
            if preds is None:
                preds = clf.predict(train_x)
            else:
                preds += clf.predict(train_x)
        preds = (preds >= ((len(train_x_s)+1)/2)).astype('int32')
        reward = evaluate_auc_prc(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
        print(reward)
    print 'best under-sampling ratio for {}:{}'.format(name, max_i)
    print 'best reward for {}:{}'.format(name, best_reward)

0.828574718075
0.838329125368
0.836883933829
0.836844033116
0.840668879763
0.835850127281
0.849854031059
0.830260229698
0.835669272118
0.839223193687
0.83493110255
0.835986754171
0.83752225517
0.835986754171
0.834683331402
best under-sampling ratio for EasyEnsemble:460
best reward for EasyEnsemble:0.849854031059


In [261]:
# Choose over-sampling ratio for different orver-sampling methods
# Supervised Model: DT(max_depth=4)
# Over sampling methods: Random, SMOTE, ADASYN
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.pipeline import make_pipeline
from collections import Counter

for sampler, name, time in zip([RandomOverSampler, SMOTE, ADASYN],
                         ['RandomOverSampler', 'SMOTE', 'ADASYN'], [10, 10, 10]):
    max_i = 1
    best_reward = -1
    for i in np.arange(1, 10, 1):
        sample = sampler(ratio={1:int(i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict_proba(train_x)[:, 1]
        reward = evaluate_auc_prc(train_y, preds)
        if best_reward < reward:
            best_reward = reward    
            max_i = i
    print 'best over-sampling ratio for {}:{}'.format(name, max_i)
    best_reward = 0
    bset_set = None
    for i in np.arange(time):
        sample = sampler(ratio={1:int(max_i*pos_num)})
        train_x_s, train_y_s = sample.fit_sample(train_x, train_y)
        clf.fit(train_x_s, train_y_s)
        preds = clf.predict_proba(train_x)[:, 1]
        reward = evaluate_auc_prc(train_y, preds)
        if best_reward < reward:
            best_reward = reward   
            bset_set = (train_x_s, train_y_s)
    print 'best reward with {}:{}'.format(name, best_reward)
    pickle.dump({'x':bset_set[0], 'y':np.eye(2)[bset_set[1].astype('int32')]}, open('../data/real/creditcard/train_{}.pkl'.format(name), 'wb+'))
    

best over-sampling ratio for RandomOverSampler:2
best reward with RandomOverSampler:0.861420563959
best over-sampling ratio for SMOTE:1
best reward with SMOTE:0.848909682323
best over-sampling ratio for ADASYN:1
best reward with ADASYN:0.848909682323


In [271]:
train_y_s

array([0, 0, 0, ..., 0, 0, 0])